In [3]:
import spacy
from fastprogress import *
nlp = spacy.load('en_core_web_sm')

In [36]:
from tqdm import tqdm_notebook, tqdm

In [22]:
from concurrent.futures import ProcessPoolExecutor
import re

def parallel(func, arr, max_workers=4):
    if max_workers<2: results = list(progress_bar(map(func, enumerate(arr)), total=len(arr)))
    else:
        with ProcessPoolExecutor(max_workers=max_workers) as ex:
            return list(progress_bar(ex.map(func, enumerate(arr)), total=len(arr)))
    if any([o is not None for o in results]): return results
    
def listify(o):
    if o is None: return []
    if isinstance(o, list): return o
    if isinstance(o, str): return [o]
    if isinstance(o, Iterable): return list(o)
    return [o]

def compose(x, funcs, *args, order_key='_order', **kwargs):
    key = lambda o: getattr(o, order_key, 0)
    for f in sorted(listify(funcs), key=key): x = f(x, **kwargs)
    return x


In [20]:
class TokenizeProcessor():
    def __init__(self, nlp, chunksize=2000, pre_rules=None, post_rules=None, max_workers=4): 
        self.chunksize,self.max_workers = chunksize,max_workers
        self.tokenizer = nlp.tokenizer
        self.pre_rules  = pre_rules 
        self.post_rules = post_rules

    def proc_chunk(self, args):
        i,chunk = args
        chunk = [compose(t, self.pre_rules) for t in chunk]
        docs = [[d.text for d in doc] for doc in self.tokenizer.pipe(chunk)]
        docs = [compose(t, self.post_rules) for t in docs]
        return docs

    def __call__(self, items): 
        toks = []
        chunks = [items[i: i+self.chunksize] for i in (range(0, len(items), self.chunksize))]
        toks = parallel(self.proc_chunk, chunks, max_workers=self.max_workers)
        return sum(toks, [])

    def proc1(self, item): return self.proc_chunk([item])[0]
 
    def deprocess(self, toks): return [self.deproc1(tok) for tok in toks]
    def deproc1(self, tok):    return " ".join(tok)

In [76]:
def multi_thread_based_tokenizations(nlp, text_list, batch_size=1000, n_threads=4, n_process=1):
    docs = nlp.pipe(text_list, batch_size=batch_size, n_threads = n_threads, n_process=n_process)
    word_sequences = []

    for doc in tqdm(docs):
        word_seq = []
        for token in doc:
            word_seq.append(token.text)
        word_sequences.append(word_seq)
    return word_sequences

In [9]:
texts = ["This is a text", "These are lots of texts"]

In [24]:
tp = TokenizeProcessor(nlp)

In [25]:
tp(texts)

[['This', 'is', 'a', 'text'], ['These', 'are', 'lots', 'of', 'texts']]

In [43]:
word_sequences = multi_thread_based_tokenizations(nlp, texts)
word_sequences

2it [00:00, 101.57it/s]


[['This', 'is', 'a', 'text'], ['These', 'are', 'lots', 'of', 'texts']]

### Example Test

In [26]:
import pandas as pd
df = pd.read_csv('/Users/ankur.kumar/Desktop/Work/projects/client/pantaloon/Pantaloon-MH-processed.csv')[['UID', 'Cleaned_Verbatim']]

In [44]:
df = df.drop_duplicates(['UID'])

In [45]:
df = df[0:10000]
df.shape

(10000, 2)

In [46]:
df['Cleaned_Verbatim'] = df['Cleaned_Verbatim'].astype(str)

In [47]:
Verbatims = df.Cleaned_Verbatim.values.tolist()

In [70]:
nlp.pipe?

with multi threading

In [64]:
%%time
word_sequences = multi_thread_based_tokenizations(nlp, Verbatims, 4)
len(word_sequences)




0it [00:00, ?it/s]


1it [00:04,  4.39s/it]


1001it [00:08,  3.07s/it]


2001it [00:13,  2.15s/it]


3001it [00:17,  1.51s/it]


4001it [00:22,  1.06s/it]


5001it [00:27,  1.35it/s]


6001it [00:31,  1.92it/s]


7001it [00:35,  2.74it/s]


8001it [00:39,  3.89it/s]


10000it [00:43, 227.69it/s]A

CPU times: user 36.9 s, sys: 7 s, total: 43.9 s
Wall time: 43.9 s


10000

In [74]:
%%time
word_sequences = multi_thread_based_tokenizations(nlp, Verbatims, 2, 6)
len(word_sequences)




0it [00:00, ?it/s]


1it [00:06,  6.53s/it]


96it [00:06,  4.57s/it]


215it [00:06,  3.20s/it]


343it [00:06,  2.24s/it]


477it [00:06,  1.57s/it]


601it [00:07,  1.10s/it]


753it [00:07,  1.30it/s]


900it [00:07,  1.86it/s]


1027it [00:07,  2.65it/s]


1169it [00:07,  3.79it/s]


1312it [00:07,  5.40it/s]


1476it [00:07,  7.71it/s]


1616it [00:07, 10.98it/s]


1785it [00:07, 15.64it/s]


1938it [00:07, 22.25it/s]


2088it [00:08, 31.48it/s]


2222it [00:08, 44.51it/s]


2356it [00:08, 62.64it/s]


2498it [00:08, 87.82it/s]


2633it [00:08, 122.04it/s]


2776it [00:08, 168.17it/s]


2920it [00:08, 228.78it/s]


3059it [00:08, 293.70it/s]


3183it [00:08, 378.70it/s]


3305it [00:09, 471.11it/s]


3423it [00:09, 566.53it/s]


3538it [00:09, 634.90it/s]


3668it [00:09, 749.19it/s]


3782it [00:09, 829.10it/s]


3895it [00:09, 884.65it/s]


4006it [00:09, 834.91it/s]


4106it [00:09, 876.64it/s]


4218it [00:10, 935.49it/s]


4328it [00:10, 978.46it/s]


4439it [00:10, 1014.

CPU times: user 7.83 s, sys: 603 ms, total: 8.43 s
Wall time: 15.2 s


10000

with multi processing

In [66]:
%%time
tp = TokenizeProcessor(nlp, chunksize=2000, max_workers=4)
word_sequences = tp(Verbatims)
len(word_sequences)

CPU times: user 3.93 s, sys: 600 ms, total: 4.53 s
Wall time: 5.47 s


10000

In [68]:
word_sequences[0:10]

[['1',
  '.',
  'the',
  'fit',
  '&',
  'price',
  'was',
  'good',
  '.',
  '2',
  '.',
  'i',
  'was',
  'not',
  'looking',
  'for',
  'any',
  'particular',
  'brand',
  'or',
  'product',
  '.',
  '3',
  '.',
  'this',
  'is',
  'was',
  'just',
  'a',
  'casual',
  'shopping',
  '.',
  '4',
  '.',
  'the',
  'staff',
  'did',
  'assist',
  'me',
  'well',
  '.',
  '5',
  '.',
  'apart',
  'form',
  'pantaloons',
  'i',
  'also',
  'shop',
  'west',
  '-',
  'side6',
  '.',
  'recommend',
  'pantaloons',
  'to',
  'friends',
  '&',
  'family',
  '-',
  'yes'],
 ['1',
  '.',
  'i',
  'am',
  'satisfied',
  'with',
  'the',
  'staff',
  'service',
  '.',
  '2',
  '.',
  'the',
  'staff',
  'was',
  'very',
  'helpful',
  'to',
  'me',
  '.',
  '3',
  '.',
  'they',
  'explained',
  'me',
  'about',
  'the',
  'products',
  '.',
  '4',
  '.',
  'recommend',
  'pantaloons',
  'to',
  'friends',
  '&',
  'family',
  '-',
  'yes'],
 ['1',
  '.',
  'i',
  'am',
  'fine',
  'with',
  'th

## Stats

In [83]:
%%time
word_sequences = multi_thread_based_tokenizations(nlp, Verbatims[0:1000], 4)




0it [00:00, ?it/s]


9it [00:00, 89.16it/s]


25it [00:00, 100.34it/s]


41it [00:00, 110.09it/s]


57it [00:00, 120.00it/s]


73it [00:00, 125.86it/s]


89it [00:00, 132.80it/s]


102it [00:00, 127.94it/s]


115it [00:00, 125.04it/s]


129it [00:00, 123.23it/s]


145it [00:01, 126.83it/s]


161it [00:01, 128.93it/s]


177it [00:01, 135.06it/s]


193it [00:01, 134.16it/s]


209it [00:01, 136.02it/s]


223it [00:01, 135.03it/s]


237it [00:01, 124.44it/s]


253it [00:01, 129.56it/s]


269it [00:02, 131.47it/s]


285it [00:02, 132.13it/s]


301it [00:02, 131.80it/s]


317it [00:02, 132.64it/s]


333it [00:02, 138.50it/s]


349it [00:02, 138.48it/s]


365it [00:02, 141.40it/s]


381it [00:02, 145.86it/s]


397it [00:02, 147.60it/s]


413it [00:03, 140.93it/s]


429it [00:03, 138.54it/s]


445it [00:03, 137.75it/s]


459it [00:03, 136.49it/s]


473it [00:03, 127.00it/s]


489it [00:03, 129.83it/s]


505it [00:03, 132.10it/s]


521it [00:03, 134.94it/s]


537it [00:03, 138.55it/s]


553i

CPU times: user 6.85 s, sys: 416 ms, total: 7.26 s
Wall time: 7.24 s


In [82]:
import time
start = time.time()
word_sequences = multi_thread_based_tokenizations(nlp, Verbatims[0:1000], 4)
end = time.time()
print(end - start)




0it [00:00, ?it/s]


13it [00:00, 111.63it/s]


29it [00:00, 122.09it/s]


49it [00:00, 130.91it/s]


65it [00:00, 135.32it/s]


81it [00:00, 140.72it/s]


97it [00:00, 136.07it/s]


113it [00:00, 132.40it/s]


129it [00:00, 135.34it/s]


145it [00:01, 138.20it/s]


161it [00:01, 138.03it/s]


177it [00:01, 143.55it/s]


193it [00:01, 142.22it/s]


213it [00:01, 150.30it/s]


229it [00:01, 138.64it/s]


245it [00:01, 138.43it/s]


261it [00:01, 141.28it/s]


277it [00:01, 142.51it/s]


293it [00:02, 143.54it/s]


309it [00:02, 143.87it/s]


325it [00:02, 140.13it/s]


341it [00:02, 139.76it/s]


356it [00:02, 141.73it/s]


371it [00:02, 142.87it/s]


386it [00:02, 143.68it/s]


401it [00:02, 142.16it/s]


417it [00:02, 140.70it/s]


433it [00:03, 140.29it/s]


449it [00:03, 141.81it/s]


465it [00:03, 142.64it/s]


481it [00:03, 143.58it/s]


497it [00:03, 144.77it/s]


513it [00:03, 147.16it/s]


529it [00:03, 142.47it/s]


549it [00:03, 146.57it/s]


564it [00:03, 145.39it/s]


57

6.987317085266113


In [ ]:
results = pd.DataFrame({''})

In [86]:
def get_stats_for_pipe_multi_threading(nlp, Verbatims, batch_sizes, n_threadss, n_processs):
    result_df = pd.DataFrame({"batch_size":[],'n_threads':[],'n_process':[],'Time':[]})
    for batch_size in batch_sizes:
        for n_threads in n_threadss:
            for n_process in n_processs:
                start = time.time()
                word_sequences = multi_thread_based_tokenizations(nlp, Verbatims, batch_size=batch_size, n_threads=n_threads, n_process=n_process)
                end = time.time()
                t_time = end - start
                result_df = result_df.append(pd.DataFrame({"batch_size":[batch_size],'n_threads':[n_threads],'n_process':[n_process],'Time':[t_time]}))
                print('result shape:', result_df.shape)
    return result_df
                

In [108]:
batch_sizes=[1000, 1500, 2000, 2500, 3000, 3500, 4000]
n_threads=[2]
n_process=[2,4]
threading_result_df = get_stats_for_pipe_multi_threading(nlp, Verbatims, batch_sizes, n_threads, n_process)





0it [00:00, ?it/s]



1it [00:04,  4.68s/it]



138it [00:04,  3.28s/it]



287it [00:04,  2.29s/it]



437it [00:04,  1.61s/it]



581it [00:05,  1.12s/it]



736it [00:05,  1.27it/s]



898it [00:05,  1.81it/s]



1033it [00:05,  2.59it/s]



1167it [00:05,  3.70it/s]



1305it [00:05,  5.27it/s]



1457it [00:05,  7.52it/s]



1594it [00:05, 10.72it/s]



1764it [00:05, 15.28it/s]



1923it [00:06, 21.73it/s]



2073it [00:09, 25.68it/s]



2197it [00:09, 36.36it/s]



2328it [00:09, 51.34it/s]



2477it [00:09, 72.27it/s]



2604it [00:09, 100.77it/s]



2743it [00:09, 139.60it/s]



2896it [00:09, 191.91it/s]



3033it [00:10, 244.31it/s]



3162it [00:10, 322.77it/s]



3301it [00:10, 419.31it/s]



3440it [00:10, 530.18it/s]



3570it [00:10, 629.35it/s]



3711it [00:10, 754.40it/s]



3844it [00:10, 866.83it/s]



3985it [00:10, 979.40it/s]



4119it [00:14, 117.00it/s]



4251it [00:14, 161.02it/s]



4390it [00:14, 219.15it/s]



4530it [00:14, 293.37it/s]



4654it [00:

result shape: (1, 4)






1it [00:05,  5.42s/it]



133it [00:05,  3.80s/it]



268it [00:05,  2.66s/it]



420it [00:05,  1.86s/it]



570it [00:05,  1.30s/it]



710it [00:05,  1.10it/s]



861it [00:06,  1.57it/s]



1001it [00:06,  2.24it/s]



1136it [00:06,  3.19it/s]



1263it [00:06,  4.55it/s]



1388it [00:06,  6.49it/s]



1541it [00:06,  9.26it/s]



1679it [00:06, 13.19it/s]



1843it [00:06, 18.78it/s]



1986it [00:06, 26.68it/s]



2129it [00:07, 37.59it/s]



2262it [00:07, 53.06it/s]



2389it [00:07, 74.41it/s]



2515it [00:07, 103.63it/s]



2640it [00:07, 142.74it/s]



2774it [00:07, 194.94it/s]



2924it [00:07, 263.76it/s]



3058it [00:07, 331.65it/s]



3180it [00:07, 424.29it/s]



3300it [00:08, 517.77it/s]



3420it [00:08, 623.91it/s]



3537it [00:08, 700.54it/s]



3654it [00:08, 796.27it/s]



3770it [00:08, 878.06it/s]



3892it [00:08, 958.07it/s]



4009it [00:11, 135.10it/s]



4127it [00:11, 183.96it/s]



4258it [00:11, 247.87it/s]



4387it [00:11, 327.00it/s]



451

result shape: (2, 4)






1it [00:07,  7.41s/it]



160it [00:07,  5.19s/it]



328it [00:07,  3.63s/it]



486it [00:07,  2.54s/it]



655it [00:07,  1.78s/it]



815it [00:07,  1.25s/it]



974it [00:08,  1.15it/s]



1153it [00:08,  1.64it/s]



1307it [00:08,  2.34it/s]



1460it [00:08,  3.34it/s]



1609it [00:08,  4.76it/s]



1770it [00:08,  6.79it/s]



1916it [00:08,  9.68it/s]



2066it [00:08, 13.79it/s]



2234it [00:08, 19.63it/s]



2386it [00:08, 27.87it/s]



2532it [00:09, 39.48it/s]



2699it [00:09, 55.84it/s]



2859it [00:09, 78.58it/s]



3013it [00:14, 52.63it/s]



3123it [00:14, 73.68it/s]



3250it [00:14, 102.68it/s]



3406it [00:14, 142.66it/s]



3544it [00:14, 195.12it/s]



3691it [00:14, 263.73it/s]



3826it [00:15, 342.68it/s]



3961it [00:15, 441.39it/s]



4111it [00:15, 559.66it/s]



4252it [00:15, 682.93it/s]



4392it [00:15, 806.79it/s]



4531it [00:16, 444.82it/s]



4662it [00:16, 554.70it/s]



4811it [00:16, 683.12it/s]



4965it [00:16, 819.04it/s]



5102it

...

result shape: (12, 4)






1it [00:19, 19.27s/it]



134it [00:19, 13.49s/it]



274it [00:19,  9.44s/it]



412it [00:19,  6.61s/it]



549it [00:19,  4.63s/it]



668it [00:19,  3.24s/it]



819it [00:19,  2.27s/it]



950it [00:19,  1.59s/it]



1107it [00:20,  1.11s/it]



1259it [00:20,  1.29it/s]



1434it [00:20,  1.84it/s]



1605it [00:20,  2.62it/s]



1761it [00:20,  3.74it/s]



1937it [00:20,  5.34it/s]



2098it [00:20,  7.62it/s]



2262it [00:20, 10.86it/s]



2422it [00:20, 15.46it/s]



2578it [00:20, 21.99it/s]



2735it [00:21, 31.23it/s]



2910it [00:21, 44.27it/s]



3071it [00:21, 62.40it/s]



3226it [00:21, 87.60it/s]



3401it [00:21, 122.50it/s]



3561it [00:21, 169.24it/s]



3722it [00:21, 231.34it/s]



3881it [00:21, 309.01it/s]



4036it [00:22, 353.24it/s]



4192it [00:22, 459.87it/s]



4350it [00:22, 583.88it/s]



4502it [00:22, 716.00it/s]



4659it [00:22, 855.43it/s]



4807it [00:22, 979.28it/s]



4957it [00:22, 1090.41it/s]



5105it [00:22, 1179.52it/s]



5253it

result shape: (13, 4)






1it [00:20, 20.25s/it]



189it [00:20, 14.18s/it]



375it [00:20,  9.92s/it]



554it [00:20,  6.95s/it]



710it [00:20,  4.86s/it]



887it [00:20,  3.40s/it]



1050it [00:20,  2.38s/it]



1230it [00:20,  1.67s/it]



1407it [00:21,  1.17s/it]



1610it [00:21,  1.22it/s]



1811it [00:21,  1.75it/s]



1995it [00:21,  2.49it/s]



2186it [00:21,  3.56it/s]



2371it [00:21,  5.08it/s]



2555it [00:21,  7.25it/s]



2755it [00:21, 10.34it/s]



2958it [00:21, 14.75it/s]



3151it [00:21, 21.00it/s]



3356it [00:22, 29.86it/s]



3552it [00:22, 42.36it/s]



3747it [00:22, 59.96it/s]



3940it [00:22, 84.21it/s]



4120it [00:22, 113.75it/s]



4307it [00:22, 158.36it/s]



4501it [00:22, 218.57it/s]



4687it [00:22, 297.22it/s]



4862it [00:23, 395.78it/s]



5036it [00:23, 512.65it/s]



5208it [00:23, 641.68it/s]



5385it [00:23, 793.26it/s]



5555it [00:23, 933.97it/s]



5723it [00:23, 1077.19it/s]



5890it [00:23, 1205.49it/s]



6058it [00:23, 1315.64it/s]



624

result shape: (14, 4)


In [110]:
threading_result_df.sort_values('Time')

,batch_size,n_threads,n_process,Time
0,1000.0,2.0,4.0,17.773555
0,1500.0,2.0,4.0,18.986561
0,2000.0,2.0,4.0,20.790800
0,2500.0,2.0,4.0,21.243879
0,3000.0,2.0,4.0,21.849630
0,3500.0,2.0,4.0,22.805901
0,1000.0,2.0,2.0,25.049998
0,4000.0,2.0,4.0,26.027611
0,2500.0,2.0,2.0,26.379638
0,1500.0,2.0,2.0,26.804577


In [92]:
def get_stats_for_concurrent(nlp, Verbatims, batch_sizes, n_processs):
    result_df = pd.DataFrame({"batch_size":[],'n_threads':[],'n_process':[],'Time':[]})
    for batch_size in batch_sizes:
            for n_process in n_processs:
                start = time.time()
                tp = TokenizeProcessor(nlp, chunksize=batch_size, max_workers=n_process)
                word_sequences = tp(Verbatims)
                end = time.time()
                t_time = end - start
                result_df = result_df.append(pd.DataFrame({"batch_size":[batch_size],'n_threads':[n_threads],'n_process':[n_process],'Time':[t_time]}))
                print('result shape:', result_df.shape)
    return result_df
                

In [106]:
batch_sizes=[1000, 1500, 2000, 2500, 3000, 3500, 4000]
n_process=[2,4]
multiprocessing_result_df_2 = get_stats_for_concurrent(nlp, Verbatims, batch_sizes, n_process)

result shape: (1, 4)


result shape: (2, 4)


result shape: (3, 4)


result shape: (4, 4)


result shape: (5, 4)


...

result shape: (10, 4)


result shape: (11, 4)


result shape: (12, 4)


result shape: (13, 4)


result shape: (14, 4)


In [112]:
del multiprocessing_result_df_2['n_threads']

In [113]:
multiprocessing_result_df_2.sort_values('Time')

,batch_size,n_process,Time
0,4000.0,2.0,3.468632
0,4000.0,4.0,3.487540
0,3500.0,2.0,4.040751
0,3000.0,2.0,4.352866
0,3000.0,4.0,4.422525
0,2500.0,4.0,4.445214
0,2500.0,2.0,4.512357
0,3500.0,4.0,4.737216
0,2000.0,4.0,5.303331
0,2000.0,2.0,5.378104
